In [96]:
library(xts); library(quantmod); library(plyr)
library("vars")
library(TTR)



read in data

In [97]:
data.env <- readRDS("WorldMarkts99_20.RDS")
# save to csv



In [98]:
markets <- ls(data.env)
markets

[1] "BSESN" "BVSP"  "FTSE"  "GDAXI" "GSPC"  "HSCE"  "IBEX"  "JKSE"  "MXX"  
[10] "N225"  "TWII"  "VIX"   "VLIC"

loop through each market and calc the returns

In [112]:

returns <- xts() # xts object is a data structure that is used to store time series data
per<- "weekly" 
for(i in seq_along(markets)) {
  sym <- markets[i]
  print(sym)
  returns <- merge(returns, periodReturn(Ad(get(sym,envir=data.env)),period=per,type = "log", dropNa = FALSE)) # this function calculates the log returns of the adjusted closing prices
  # add a column header to the new column
  colnames(returns)[ncol(returns)] <- sym
}

[1] "BSESN"


Warning message in to_period(xx, period = on.opts[[period]], ...):
"missing values removed from data"


[1] "BVSP"


Warning message in to_period(xx, period = on.opts[[period]], ...):
"missing values removed from data"


[1] "FTSE"


...

[1] "N225"


Warning message in to_period(xx, period = on.opts[[period]], ...):
"missing values removed from data"


[1] "TWII"


Warning message in to_period(xx, period = on.opts[[period]], ...):
"missing values removed from data"


[1] "VIX"
[1] "VLIC"


In [113]:
# save daily returns to csv
# include index
write.zoo(returns, file = "data/weekly_returns.csv", index.name = "Date", sep = ",")

impute values

In [100]:

returns_imputed <- returns

returns_imputed <- na.locf(returns_imputed, fromLast=TRUE)

# count na values
sum(is.na(returns_imputed))

[1] 12

In [101]:
# it is the last roww that has na values so we can remove it
returns_imputed <- returns_imputed[-nrow(returns_imputed),]
sum(is.na(returns_imputed))

[1] 0

treat outliers

In [102]:
# find outliers and impute them with locf

# find outliers
outliers <- function(x) {
  qnt <- quantile(x, probs=c(.25, .75), na.rm = TRUE)
  H <- 1.75 * IQR(x, na.rm = TRUE)
  y <- x
  y[x < (qnt[1] - H)] <- NA
  y[x > (qnt[2] + H)] <- NA
  return(y)
}

returns_imputed <- apply(returns_imputed, 2, outliers)


In [105]:
# impute outliers 

returns_imputed <- na.locf(returns_imputed, fromLast=TRUE)
returns_imputed

,BSESN,BVSP,FTSE,GDAXI,GSPC,HSCE,IBEX,JKSE,MXX,N225,TWII,VIX,VLIC
1999-01-04 01:00:00,0.000000000,0.000000000,0.000000e+00,0.000000000,0.000000000,0.000000000,0.000000e+00,0.000000000,0.0000000000,0.0000000000,0.000000000,0.000000000,0.0000000000
1999-01-05 01:00:00,0.008617368,0.024052963,1.331376e-02,-0.005107135,0.013490591,-0.012762232,1.923424e-02,-0.005013759,-0.0199753501,-0.0137457197,0.000000000,-0.067574627,0.0065784811
1999-01-06 01:00:00,0.017820201,0.030469051,-7.771404e-03,-0.018017626,0.021898867,0.045278372,-1.965542e-02,-0.005013759,-0.0039227700,0.0176565734,0.007687676,-0.046870463,0.0153324334
1999-01-07 01:00:00,0.028722484,-0.025192450,-7.771404e-03,-0.018017626,-0.002053434,0.013437532,-1.965542e-02,-0.005013759,-0.0215650976,0.0050434885,0.002719466,0.043184282,-0.0042317757
1999-01-08 01:00:00,0.029493262,-0.025192450,7.511220e-03,0.004628470,0.004212474,-0.014236682,-2.982459e-03,-0.005013759,-0.0073112817,-0.0107508474,0.002719466,-0.045758243,0.0044998431
1999-01-11 01:00:00,0.010355711,0.036985035,-1.017000e-02,-0.019562483,-0.008830379,-0.036626939,-1.584672e-02,-0.005013759,-0.0128204496,-0.0017435616,-0.002301057,0.089513888,-0.0040511643
1999-01-12 01:00:00,-0.023574603,0.036985035,-8.482863e-03,-0.013446254,-0.019470210,-0.042497874,-2.350646e-02,-0.002532431,0.0168758944,-0.0005619825,-0.006749797,0.098661022,-0.0143438968
1999-01-13 01:00:00,-0.012833929,0.036985035,-5.124113e-03,-0.016059255,-0.004131107,-0.007389931,-1.879106e-03,-0.025495065,0.0168758944,0.0031855486,-0.007025049,0.069087799,-0.0072842251
1999-01-14 01:00:00,-0.005506798,0.036985035,-5.124113e-03,-0.016059255,-0.018156449,-0.007389931,-1.879106e-03,-0.025495065,0.0168758944,0.0247050411,-0.012423046,0.091043941,-0.0142645050
1999-01-15 01:00:00,-0.025979381,0.036985035,2.054281e-02,0.014383799,0.007005303,-0.001956264,3.091169e-02,-0.015172450,0.0055130426,0.0048068209,-0.012056146,-0.120363682,0.0179018225


extracting epoch time from the date and time columns

In [104]:
dI="2001-01-01"; dF="2003-01-01"
Retp <- returns_imputed[paste(dI,"/",dF,sep=""),]

ERROR: Error in returns_imputed[paste(dI, "/", dF, sep = ""), ]: subscript out of bounds


### Question 1

In [91]:
library(ggplot2)

In [92]:
# Set lambda value
lambda <- 0.94

for (i in 1:ncol(Retp)) {
    returns <- Retp[, i]
    # Compute EWMA-based variance using EMA function
    # by using the EMA function, we can calculate the EWMA-based variance when we set the lambda value
    # the lambda value is the weight of the most recent observation
    # the difference between EMA and EWMA is that EMA is used to calculate the average of the data points, while EWMA is used to calculate the variance of the data points
    ewma_variance <- EMA(returns^2, n = 1 / (1 - lambda)) 

    # Compare with other volatility estimations (e.g., historical volatility)
    historical_volatility <- sqrt(rollapply(returns^2, width = 21, FUN = sum, align = "right", fill = NA) / 21)

    # 

    ggplot( aes(x = historical_volatility, y = index(historical_volatility))) +
    geom_line() +
    labs(x = "Date", y = "Value", title = "Line Graph Example")


}

ERROR: [1m[33mError[39m in `fortify()`:[22m
[1m[22m[33m![39m `data` must be a [34m<data.frame>[39m, or an object coercible by `fortify()`,
  or a valid [34m<data.frame>[39m-like object coercible by `as.data.frame()`, not a
  <uneval> object.
[36mi[39m Did you accidentally pass `aes()` to the `data` argument?
